In [1]:
!pip install openai
!pip install langchain
!pip install langchain-openai
!pip install docarray

Ok, so far we've discussed the basics of agents, and looked at practical examples of implementing them in Python, from a naive approach where we give tools inside the prompt to the model and ask it to generate function calls from there, to using frameworks like openai or langchain in combination with function calling to better optimize and struture these calls to the necessary tools. 

Now, let's look at in practice how to put things together to build cool agents that can perform interesting and productive tasks.

The framework we'll use to do that is [LangChain](https://python.langchain.com/docs/get_started/introduction).

So, before we dive into agents, let's quickly take a look at this framework to understand how it allows for building these complex functionalities.

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [1]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [9]:
from langchain.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI

# os.environ["OPENAI_API_KEY"]=""
chat_model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo-1106")

You can predict outputs from both LLMs and ChatModels:

In [10]:
chat_model.invoke("hi!")
# Output: "Hi"

AIMessage(content='Hello! How can I help you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [11]:
chat_model.invoke("What is the best part about using LLMs with tools?")

AIMessage(content='The best part about using LLMs (Language Model Models) with tools is their ability to understand and generate human-like language. This can be incredibly useful for tasks such as natural language processing, text generation, and conversation modeling. LLMs can help improve the accuracy and effectiveness of these tools, making them more useful for a wide range of applications. Additionally, LLMs can be fine-tuned for specific tasks, making them versatile and adaptable for different use cases.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

Basic components are:

- Models
- Prompt templates
- Output parsers

In [12]:
from langchain.prompts import (
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain.schema.output_parser import StrOutputParser

Comprehensive example:

In [15]:
# Define the system message
system_template = """You are a helpful AI assistant who is weirdly obsessed with pancakes. 
Your role is to provide helpful information to users, but you always make a random comment
at the end about pancakes.
"""

In [16]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [17]:
# Define the human message 
human_template = "Hello, {subject}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Combine the system and human messages into a chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# Format the chat prompt with a subject
formatted_prompt = chat_prompt.format_prompt(subject="AI!")
print(formatted_prompt)

messages=[SystemMessage(content='You are a helpful AI assistant who is weirdly obsessed with pancakes. \nYour role is to provide helpful information to users, but you always make a random comment\nat the end about pancakes.\n'), HumanMessage(content='Hello, AI!')]


Template Example

In [18]:
prompt = ChatPromptTemplate.from_template("Show me 5 examples of this concept: {concept}")

In [19]:
prompt

ChatPromptTemplate(input_variables=['concept'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['concept'], template='Show me 5 examples of this concept: {concept}'))])

In [20]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [21]:
prompt.format(concept="animal")

'Human: Show me 5 examples of this concept: animal'

In [22]:
chain = prompt | chat_model

output = chain.invoke({"concept": "animal"})
output

AIMessage(content='1. A tiger is an example of an animal that belongs to the feline family.\n2. A dolphin is an example of an animal that is highly intelligent and lives in water.\n3. A bald eagle is an example of an animal that is a symbol of strength and freedom in the United States.\n4. A giraffe is an example of an animal with a long neck and legs, adapted for browsing on tall trees.\n5. A chimpanzee is an example of an animal that is closely related to humans and exhibits complex social behaviors.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [23]:
from IPython.display import Markdown

Markdown(output.content)

1. A tiger is an example of an animal that belongs to the feline family.
2. A dolphin is an example of an animal that is highly intelligent and lives in water.
3. A bald eagle is an example of an animal that is a symbol of strength and freedom in the United States.
4. A giraffe is an example of an animal with a long neck and legs, adapted for browsing on tall trees.
5. A chimpanzee is an example of an animal that is closely related to humans and exhibits complex social behaviors.

In [24]:
chat_model.invoke("I am teaching a live-training about LLMs!")

AIMessage(content="Great! LLMs, or Language Model LMs, are a fascinating and powerful tool in natural language processing. They have revolutionized the way we approach language understanding and generation.\n\nIn this live-training, we will cover the basics of LLMs, including what they are, how they work, and their applications in various fields such as machine translation, chatbots, and text generation. We will also discuss different types of LLMs, such as GPT-3, BERT, and Transformer, and their specific features and capabilities.\n\nWe will also explore best practices for training and fine-tuning LLMs, as well as common challenges and considerations when working with these models. Additionally, we will discuss ethical considerations and potential biases in LLMs, and how to mitigate them.\n\nBy the end of this live-training, you will have a solid understanding of LLMs and their potential impact on the future of natural language processing. You will also have the knowledge and tools to

In [25]:
from langchain.schema import HumanMessage

text = "What would be a good dog name for a dog that loves to nap?"
messages = [HumanMessage(content=text)]

chat_model.invoke(messages)

AIMessage(content='"Snooze"', response_metadata={'finish_reason': 'stop', 'logprobs': None})

At this point let's stop and take a look at what this code would look like if we were using the openai api directly instead.

Let's understand what is going on.

Instead of writing down the human message dictionary for the openai API as you would do normally using the the original API, langchain is giving you an abstraction over that message through the class
`HumanMessage()`, as well as an abstraction over the loop for multiple predictions through the .`invoke()` method.

Now, why is that an useful thing?

Because it allows you to work at a higher level of experimentation and orchestration with the blocks of that make up a workflow using LLMs.

By making it easier to create predictions of multiple messages for example, you can experiment with different human message prompts faster and therefore get to better and more efficient results faster without having to write a lot of boilerplate.

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [14]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'What is a good dog name for a dog that loves to sleeping?'

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [27]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# Output: ['hi', 'bye']

['hi', 'bye']

In [29]:
from langchain.schema import StrOutputParser

print(StrOutputParser().parse(output))

content='1. A tiger is an example of an animal that belongs to the feline family.\n2. A dolphin is an example of an animal that is highly intelligent and lives in water.\n3. A bald eagle is an example of an animal that is a symbol of strength and freedom in the United States.\n4. A giraffe is an example of an animal with a long neck and legs, adapted for browsing on tall trees.\n5. A chimpanzee is an example of an animal that is closely related to humans and exhibits complex social behaviors.' response_metadata={'finish_reason': 'stop', 'logprobs': None}


This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

In [31]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
)
chain.invoke("sleep")

{'activity': 'sleep', 'text': 'Snooze'}

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [32]:
chain = prompt | ChatOpenAI()

chain.invoke({"activity": "sleep"})

AIMessage(content='Snooze', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [34]:
chain = prompt | ChatOpenAI() | StrOutputParser()

chain.invoke({"activity": "sleep"})

'Snooze'

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

In [35]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from IPython.display import Markdown


model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_template(template="Name 5 concepts related to this: {concept}. The output should be in bullet points.")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

Markdown(chain.invoke({"concept": "probability distribution"}))

- Normal distribution
- Binomial distribution
- Poisson distribution
- Exponential distribution
- Uniform distribution

# LangChain Lab Exercises

Let's take a look at a few examples using some of the capabilities of the LangChain library.

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

In [19]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [20]:
llm.invoke("Hi! Tell me a joke about an instructor who is so childish, he makes examples where he calls himself the agent-master")

AIMessage(content="Why did the childish instructor call himself the agent-master? Because he couldn't resist the urge to play spy games in class!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [21]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [22]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [23]:
from langchain.tools.render import format_tool_to_openai_function

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

/Users/greatmaster/miniconda3/envs/oreilly-agents-test-env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [24]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [25]:
agent.invoke({
    "input": "how many letters in the word educa?",
    "intermediate_steps": []
})

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"word":"educa"}', 'name': 'get_word_length'}}, response_metadata={'finish_reason': 'function_call', 'logprobs': None})])

In [26]:
from langchain.schema.agent import AgentFinish

intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "how many letters in the word educa?",
        "intermediate_steps": intermediate_steps
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        tool = {
            "get_word_length": get_word_length
        }[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

get_word_length {'word': 'educa'}
There are 5 letters in the word "educa".


In [27]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
agent_executor.invoke({"input": "how many letters in the word educa?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'how many letters in the word educa?',
 'output': 'There are 5 letters in the word "educa".'}

In [29]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [30]:
from langchain.schema.messages import HumanMessage, AIMessage
chat_history = []

In [31]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.append(HumanMessage(content=input1))
chat_history.append(AIMessage(content=result['output']))
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
"Educa" is not a common English word. It seems to be a shortened form of the word "education".

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': '"Educa" is not a common English word. It seems to be a shortened form of the word "education".'}

In [33]:
agent_executor.invoke({"input": "How many letters in the sentence 'Hi my name is Budha Master'", "chat_history": chat_history})




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Hi my name is Budha Master'}`
responded: To calculate the number of letters in the sentence "Hi my name is Budha Master", we need to count all the letters including spaces and punctuation.

Let's calculate the total number of letters in the sentence.

26There are 26 letters in the sentence "Hi my name is Budha Master".

> Finished chain.


{'input': "How many letters in the sentence 'Hi my name is Budha Master'",
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'There are 26 letters in the sentence "Hi my name is Budha Master".'}